In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/final_R4_2/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.24708544 0.26151288 0.24788217 0.2435195 ]]
[[0.22957364 0.24811168 0.2699197  0.25239497]]
[[0.22957364 0.24811168 0.2699197  0.25239497]]
[[0.22957364 0.24811168 0.2699197  0.25239497]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR, replay_ratio=0):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", 20, 0.99, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), 20, 0.99, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [ ]:
tf.reset_default_graph()
start_method(replay_ratio=4)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1  using  4  per online step
worker_2  using  4  per online step
worker_3  using  4  per online step
worker_4  using  4  per online step
worker_5  using  4  per online step
worker_6  using  4  per online step
worker_7  using  4  per online step
worker_8  using  4  per online step
worker_9  using  4  per online step
worker_10  using  4  per online step
worker_11  using  4  per online step
worker_12  using  4  per online step
worker_13  using  4  per online step
worker_14  using  4  per online step
worker_15  using  4  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  894  Running/Max:  1.0 1.0  Frames: 12
pi: [[0.2466458  0.25768873 0.2466055  0.24905999]]
Saving Model
Worker  worker_eval At  10182  Running/Max:  1.0694105110000005 2.0  Frames: 129
pi: [[0.17870012 0.27505532 0.27116928 0.27507538]]
Saving Model
Worker  worker_eval At  20496  Running/Max:  1.1488893788020245 4.0  Frames: 259
pi: [[0.18893696 0.28876117 0.25678813 0.26551363]]
Saving Model
Worker  worker_eval At  30793  Running/Max:  1.9552544754915409 4.0  Frames: 393
pi: [[0.22667025 0.28166232 0.2520195  0.23964791]]
Saving Model
Worker  worker_eval At  41583  Running/Max:  2.4278345189048576 6.0  Frames: 531
pi: [[0.22099356 0.2889877  0.19900365 0.2910151 ]]
Saving Model
Worker  worker_eval At  51003  Running/Max:  2.3219984216946674 6.0  Frames: 654
pi: [[0.1816688  0.3859584  0.19226891 0.24010396]]
Saving Model
Worker  worker_eval At  60720  Running/Max:  2.237980562211308 6.0  Frames: 778
pi: [[0.22253843 0.4152955  0.2344047  0.12776148]]
Saving Mod

Worker  worker_eval At  563077  Running/Max:  18.364933301160278 28.0  Frames: 7165
pi: [[0.01528627 0.9686854  0.00105208 0.01497628]]
Saving Model
Worker  worker_eval At  573993  Running/Max:  19.625595973939827 28.0  Frames: 7306
pi: [[1.4015788e-01 8.4175771e-01 1.7922031e-02 1.6240041e-04]]
Saving Model
Worker  worker_eval At  580673  Running/Max:  18.846732738891262 28.0  Frames: 7393
pi: [[0.20311132 0.78771526 0.0008135  0.00836002]]
Saving Model
Worker  worker_eval At  592891  Running/Max:  18.358268166651733 28.0  Frames: 7551
pi: [[0.6467011  0.3202508  0.00945425 0.02359382]]
Saving Model
Worker  worker_eval At  604921  Running/Max:  20.150197214987905 32.0  Frames: 7707
pi: [[0.30854896 0.57692766 0.06203583 0.05248763]]
Saving Model
Worker  worker_eval At  612897  Running/Max:  19.511659744140207 32.0  Frames: 7808
pi: [[9.7825797e-04 2.3323159e-01 8.8125544e-06 7.6578134e-01]]
Saving Model
Worker  worker_eval At  621618  Running/Max:  19.404444392753568 32.0  Frames: 791

Worker  worker_eval At  1103337  Running/Max:  35.53058207439028 47.0  Frames: 14110
pi: [[7.9406649e-03 4.7438970e-01 8.5232416e-05 5.1758438e-01]]
Saving Model
Worker  worker_eval At  1110868  Running/Max:  36.07752386695125 47.0  Frames: 14208
pi: [[0.01531969 0.98167115 0.00176481 0.00124436]]
Saving Model
Worker  worker_eval At  1125109  Running/Max:  36.10279433223052 47.0  Frames: 14391
pi: [[0.06702264 0.3176188  0.55130583 0.06405269]]
Saving Model
Worker  worker_eval At  1133108  Running/Max:  36.69251489900748 47.0  Frames: 14492
pi: [[0.5605665  0.30856588 0.12931007 0.00155758]]
Saving Model
Worker  worker_eval At  1145317  Running/Max:  35.52093706819606 47.0  Frames: 14647
pi: [[0.11870955 0.5671435  0.30987406 0.00427287]]
Saving Model
Worker  worker_eval At  1152751  Running/Max:  35.768843361376454 47.0  Frames: 14741
pi: [[0.20006675 0.79590464 0.0021672  0.00186145]]
Saving Model
Worker  worker_eval At  1167207  Running/Max:  35.73276312271493 47.0  Frames: 14927
pi

Worker  worker_eval At  1630639  Running/Max:  38.253599573290884 57.0  Frames: 20896
pi: [[0.50364435 0.38256374 0.06227057 0.05152131]]
Saving Model
Worker  worker_eval At  1645962  Running/Max:  39.03541565436562 57.0  Frames: 21093
pi: [[0.20893982 0.08631509 0.05418188 0.65056324]]
Saving Model
Worker  worker_eval At  1653594  Running/Max:  39.63187408892906 57.0  Frames: 21192
pi: [[6.6864574e-01 2.4662144e-01 1.5755753e-04 8.4575236e-02]]
Saving Model
Worker  worker_eval At  1660296  Running/Max:  40.268686680036154 57.0  Frames: 21278
pi: [[0.7594987  0.12924154 0.10514114 0.00611861]]
Saving Model
Worker  worker_eval At  1678150  Running/Max:  41.94763621082929 57.0  Frames: 21512
pi: [[3.6236525e-02 8.8538289e-01 7.7949464e-02 4.3115887e-04]]
Saving Model
Worker  worker_eval At  1685837  Running/Max:  42.35287258974636 57.0  Frames: 21612
pi: [[1.0953724e-03 9.9819726e-01 4.4118235e-04 2.6615238e-04]]
Saving Model
Worker  worker_eval At  1695900  Running/Max:  44.517585330771

Worker  worker_eval At  2153131  Running/Max:  43.22184761897141 70.0  Frames: 27616
pi: [[2.4781518e-03 9.8938841e-01 2.2108186e-06 8.1311511e-03]]
Saving Model
Worker  worker_eval At  2161469  Running/Max:  44.49966285707427 70.0  Frames: 27724
pi: [[0.82792467 0.1658695  0.00086747 0.00533834]]
Saving Model
Worker  worker_eval At  2170038  Running/Max:  44.44969657136684 70.0  Frames: 27835
pi: [[2.3025988e-01 7.6256037e-01 8.3811506e-07 7.1788630e-03]]
Saving Model
Worker  worker_eval At  2187648  Running/Max:  46.514254222807146 70.0  Frames: 28062
pi: [[9.0869188e-01 1.0482385e-03 2.9730874e-08 9.0259895e-02]]
Saving Model
Worker  worker_eval At  2198229  Running/Max:  49.36282880052643 75.0  Frames: 28197
pi: [[7.8437971e-11 1.0000000e+00 1.8796172e-17 3.9404949e-17]]
Saving Model
Worker  worker_eval At  2204229  Running/Max:  47.826545920473784 75.0  Frames: 28274
pi: [[7.909709e-14 1.000000e+00 2.860326e-19 1.059838e-13]]
Saving Model
Worker  worker_eval At  2210200  Running/M

Worker  worker_eval At  2678198  Running/Max:  54.09082358820562 84.0  Frames: 34365
pi: [[9.9948263e-01 2.5905214e-05 2.3608084e-08 4.9152924e-04]]
Saving Model
Worker  worker_eval At  2684316  Running/Max:  51.78174122938506 84.0  Frames: 34443
pi: [[2.8382262e-06 9.9999714e-01 3.4940648e-14 7.5617845e-10]]
Saving Model
Worker  worker_eval At  2692735  Running/Max:  51.703567106446556 84.0  Frames: 34551
pi: [[1.7627601e-03 8.0642569e-01 5.5039536e-08 1.9181143e-01]]
Saving Model
Worker  worker_eval At  2700720  Running/Max:  51.6332103958019 84.0  Frames: 34652
pi: [[2.1049936e-01 7.4051768e-01 6.9173548e-05 4.8913710e-02]]
Saving Model
Worker  worker_eval At  2714643  Running/Max:  49.04290042059954 84.0  Frames: 34830
pi: [[6.4665697e-15 1.0000000e+00 3.0885967e-21 3.2093902e-14]]
Saving Model
Worker  worker_eval At  2722607  Running/Max:  48.93861037853958 84.0  Frames: 34931
pi: [[3.1629238e-01 4.4465554e-01 3.8854138e-05 2.3901325e-01]]
Saving Model
Worker  worker_eval At  2731

In [ ]:
TB_DIR = "saves/breakout/final_R2/"
tf.reset_default_graph()
start_method(replay_ratio=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1worker_2   using  2  per online step
worker_3  using  2  per online step
 using  2  per online step
worker_4  using  2  per online step
worker_5 worker_6  using  2  per online step using  2  per online step
worker_7  using  2  per online step

worker_8  using  2  per online step
worker_9  using  2  per online step
worker_10  using  2  per online step
worker_11  using  2  per online step
worker_12  using  2  per online step
worker_13  using  2  per online step
worker_14 worker_15  using  2  per online step
 using  2  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  433  Running/Max:  0.9 0.0  Frames: 10
pi: [[0.257915   0.23606363 0.25107047 0.25495088]]
Saving Model
Worker  worker_eval At  10473  Running/Max:  1.3503623110910397 5.0  Frames: 218
pi: [[0.31936288 0.14051716 0.16580871 0.37431127]]
Saving Model
Worker  worker_eval At  20120  Running/Max:  1.839359017567855 5.0  Frames: 418
pi: [[0.4172968  0.11229338 0.11469973 0.35571012]]
Saving Model
Worker  worker_eval At  30569  Running/Max:  1.4750734480906693 5.0  Frames: 637
pi: [[0.3297688  0.17505193 0.12908956 0.3660897 ]]
Saving Model
Worker  worker_eval At  40378  Running/Max:  1.669809376594986 5.0  Frames: 840
pi: [[0.05900332 0.75764287 0.09149472 0.09185905]]
Saving Model
Worker  worker_eval At  50384  Running/Max:  3.0260998262813503 6.0  Frames: 1046
pi: [[0.04416445 0.66574085 0.15592468 0.13417004]]
Saving Model
Worker  worker_eval At  60495  Running/Max:  4.319477264460738 10.0  Frames: 1256
pi: [[0.01904438 0.33477816 0.6130371  0.03314032]]
Saving Mo

In [7]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [8]:
tf.reset_default_graph()
test_method()

INFO:tensorflow:Restoring parameters from saves/breakout/final_R1/checkpoints/model1981886.cptk


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Exception in thread Thread-38:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", li